In [ ]:
import pickle

import pandas as pd

In [ ]:
# データの読み込み
df = pd.read_csv("./data/data_without_nan.tsv", sep="\t")
df

In [ ]:
# ある長さ以上の文章のみを抽出し、バッチサイズごとにデータを分割して保存する

# 最低限の文字列の長さを設定
min_length = 20
# バッチサイズを設定
batch_size = 64

# "文章"列の中身の長さがmin_length以上のデータを抽出
long_texts_df = df[df["文章"].str.len() >= min_length].copy()

# "満足度"列の値を変換
long_texts_df.loc[:, "満足度"] = long_texts_df["満足度"].apply(
    lambda x: -1 if x == -0.5 else 1 if x == 0.5 else x
)

# # "満足度"の列の値が1なら"満足"、-1なら"不満"としてラベルを付与
# long_texts_df.loc[:, "満足度"] = long_texts_df["満足度"].apply(
#     lambda x: "満足" if x == 1 else "不満"
# )

# バッチサイズごとにデータを分割
df_batches = [
    long_texts_df[i : i + batch_size] for i in range(0, len(long_texts_df), batch_size)
]

print(f"元のデータの行数: {len(df)}")
print(f"{min_length}文字以上のデータの行数: {len(long_texts_df)}")

In [ ]:
long_texts_df
# df_batches[0]

In [ ]:
# 保存
with open("./data/df_batches.pkl", "wb") as f:
    pickle.dump(df_batches, f)

In [ ]:
# 正解データを辞書型で保存

long_texts_df["満足度"] = long_texts_df["満足度"].astype(str)
# "満足度"の列の値が1なら"満足"、-1なら"不満"としてラベルを付与
long_texts_df.loc[:, "満足度"] = (
    long_texts_df["満足度"].apply(lambda x: "満足" if x == "1.0" else "不満")
)
original_data_dict = {
    index: {"label": row["ラベル"], "sentiment": row["満足度"], "text": row["文章"]}
    for index, row in long_texts_df.iterrows()
}
# original_data_dict

In [ ]:
# 保存
with open("./data/original_data_dict.pkl", "wb") as f:
    pickle.dump(original_data_dict, f)